## import packages and model tokenizer

Important note: change ImprovementContent to ImprovementName , remember to change them after generate answer

In [1]:
!pip install torch  --quiet

# # Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

# #FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
!pip install -U transformers
# # !pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

# # Uncomment only if you're using A100 GPU
# #!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet

# %pip install -U wandb

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset, DatasetDict
from trl import SFTTrainer, setup_chat_format

In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HF_TOKEN")

login(token = hf_token)

wb_token = user_secrets.get_secret("wandb")

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune Llama 3 8B on SQL dataset', 
    job_type="training", 
    anonymous="allow"
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: huyhoangt2201 (huyhoangt2201-fpt-university). Use `wandb login --relogin` to force relogin


In [4]:
base_model = "phamhai/Llama-3.2-1B-Instruct-Frog"
new_model = "llama-3.2-1b-sql_finetuned_multitableJidouka2_1.0_977_records_mix"

In [5]:
!pip install safetensors

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [6]:
torch_dtype = torch.float16

# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,

    #attn_implementation=attn_implementation
)
tokenizer = AutoTokenizer.from_pretrained(base_model,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [7]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

In [8]:
from datasets import load_dataset, DatasetDict

In [9]:
dataset_train = load_dataset("huyhoangt2201/contextawareJidouka_fixed", split='train[:95%]')
dataset_val = load_dataset("huyhoangt2201/contextawareJidouka_fixed", split='train[-5%:]')
dataset = DatasetDict({
    'train': dataset_train,
    'validation': dataset_val
})
dataset.save_to_disk("completed_train_dataset")

(…)tput152_records_contextAware_english.csv:   0%|          | 0.00/78.8k [00:00<?, ?B/s]

(…)tput157_records_contextAware_english.csv:   0%|          | 0.00/66.6k [00:00<?, ?B/s]

(…)d_time_department_dcgroup_vietnamese.csv:   0%|          | 0.00/151k [00:00<?, ?B/s]

(…)ixed_time_department_dcgroup_english.csv:   0%|          | 0.00/135k [00:00<?, ?B/s]

(…)t296_records_contextAware_vietnamese.csv:   0%|          | 0.00/197k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/977 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/928 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/49 [00:00<?, ? examples/s]

## process data

In [10]:
# system_prompt = """You are an SQL query assistant. Based on schema and context below, generate an SQL query to retrieve the relevant information for the user. If the user’s question is unrelated to the table, respond naturally in user's language.

# Schema:
# +Table Author, columns=[AuthorId: int, AuthorName: nvarchar(255), DepartmentId int, GroupDCId int]
# +Table Department, columns=[DepartmentId: int, DepartmentName: nvarchar(255)]
# +Table GroupDC, columns=[GroupDCId: int, DepartmentId: int, GroupDCName nvarchar(255)]
# +Table Job, columns=[JobId: int, JobName: nvarchar(255)]
# +Table Tool, columns=[ToolId: int, ToolName: nvarchar(255), ToolDescription: text]
# +Table Jidouka, columns=[JidoukaId: bigint, ProductApply: nvarchar(255), ImprovementName: nvarchar(255), SoftwareUsing: nvarchar(255), Description: nvarchar(255), Video: text, DetailDocument: text, TotalJobApplied: int, TotalTimeSaved: int, DateCreate: datetime, JobId: int, AuthorId: int, DepartmentId: int, GroupDCId: int]
# +Table JidoukaTool, columns=[JidoukaId: bigint, ToolId: int]
# +Primary_keys=[Author.AuthorId, Department.DepartmentId, GroupDC.GroupDCId, Job.JobId, Tool.ToolId, Jidouka.JidoukaId]
# +Foreign_keys=[GroupDC.DepartmentId=Department.DepartmentId, Jidouka.JobId=Job.JobId, Jidouka.AuthorId=Author.AuthorId, Jidouka.DepartmentId=Department.DepartmentId, Jidouka.GroupDCId=GroupDC.GroupDCId, JidoukaTool.JidoukaId=Jidouka.JidoukaId, JidoukaTool.ToolId=Tool.ToolId, Author.DepartmentId=Department.DepartmentId, Author.GroupDCId=GroupDC.GroupDCId]

# Context:
# Previous user question: {previous_question}
# Previous answer: {previous_answer}
# Previous schema linking(Format: [Tables, Columns, Foreign keys, Possible cell values]): {schema_linking}
# """

In [11]:
# def format_context(sample):
#     sample['context'] = system_prompt.format(previous_question=sample['previous_question'], previous_answer=sample['previous_answer'], schema_linking=sample['schema_linking'])
    
#     return sample

In [12]:
system_prompt = """You are an SQL query assistant. Based on schema, generate an SQL query to retrieve the relevant information for the user. If the user’s question is unrelated to the table, respond naturally in user's language.

Schema:
+Table Author, columns=[AuthorId: int, AuthorName: nvarchar(255), DepartmentId int, GroupDCId int]
+Table Department, columns=[DepartmentId: int, DepartmentName: nvarchar(255)]
+Table GroupDC, columns=[GroupDCId: int, DepartmentId: int, GroupDCName nvarchar(255)]
+Table Job, columns=[JobId: int, JobName: nvarchar(255)]
+Table Tool, columns=[ToolId: int, ToolName: nvarchar(255), ToolDescription: text]
+Table Jidouka, columns=[JidoukaId: bigint, ProductApply: nvarchar(255), ImprovementName: nvarchar(255), SoftwareUsing: nvarchar(255), Description: nvarchar(255), Video: text, DetailDocument: text, TotalJobApplied: int, TotalTimeSaved: int, DateCreate: datetime, JobId: int, AuthorId: int, DepartmentId: int, GroupDCId: int]
+Table JidoukaTool, columns=[JidoukaId: bigint, ToolId: int]
+Primary_keys=[Author.AuthorId, Department.DepartmentId, GroupDC.GroupDCId, Job.JobId, Tool.ToolId, Jidouka.JidoukaId]
+Foreign_keys=[GroupDC.DepartmentId=Department.DepartmentId, Jidouka.JobId=Job.JobId, Jidouka.AuthorId=Author.AuthorId, Jidouka.DepartmentId=Department.DepartmentId, Jidouka.GroupDCId=GroupDC.GroupDCId, JidoukaTool.JidoukaId=Jidouka.JidoukaId, JidoukaTool.ToolId=Tool.ToolId, Author.DepartmentId=Department.DepartmentId, Author.GroupDCId=GroupDC.GroupDCId]
"""

In [13]:
def format_context(sample):
    sample['context'] = system_prompt
    
    return sample

In [14]:
dataset_train2 = dataset_train.map(format_context)
dataset_val2 = dataset_val.map(format_context)

Map:   0%|          | 0/928 [00:00<?, ? examples/s]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [15]:
dataset_train2

Dataset({
    features: ['previous_question', 'previous_answer', 'schema_linking', 'question', 'answer', 'context'],
    num_rows: 928
})

In [16]:
dataset_train3 = dataset_train2.shuffle(seed=42)
dataset_val3 = dataset_val2.shuffle(seed=42)

In [17]:
def format_data_template(sample):
    chat = [
          {"role":"system", "content": sample['context']},
          {"role":"user", "content":sample['previous_question']},
          {"role":"assistant","content":sample['previous_answer']}
    ]
    return {
        "messages": tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    }

In [18]:
train_set = dataset_train3.map(format_data_template, remove_columns=['context','question','answer','previous_question', 'previous_answer','schema_linking'])
val_set = dataset_val3.map(format_data_template, remove_columns=['context','question','answer','previous_question', 'previous_answer','schema_linking'])

Map:   0%|          | 0/928 [00:00<?, ? examples/s]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [19]:
train_set['messages'][1]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Nov 2024\n\nYou are an SQL query assistant. Based on schema, generate an SQL query to retrieve the relevant information for the user. If the user’s question is unrelated to the table, respond naturally in user's language.\n\nSchema:\n+Table Author, columns=[AuthorId: int, AuthorName: nvarchar(255), DepartmentId int, GroupDCId int]\n+Table Department, columns=[DepartmentId: int, DepartmentName: nvarchar(255)]\n+Table GroupDC, columns=[GroupDCId: int, DepartmentId: int, GroupDCName nvarchar(255)]\n+Table Job, columns=[JobId: int, JobName: nvarchar(255)]\n+Table Tool, columns=[ToolId: int, ToolName: nvarchar(255), ToolDescription: text]\n+Table Jidouka, columns=[JidoukaId: bigint, ProductApply: nvarchar(255), ImprovementName: nvarchar(255), SoftwareUsing: nvarchar(255), Description: nvarchar(255), Video: text, DetailDocument: text, TotalJobApplied: int, TotalTimeSaved: int

## Training

In [20]:
early_stopping_callback = EarlyStoppingCallback( 
    early_stopping_patience=5
)

training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="adamw_8bit",
    num_train_epochs=30,
    eval_strategy="epoch",
    eval_steps=0.2,
    save_strategy='epoch',
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    group_by_length=True,
    report_to="wandb",
    load_best_model_at_end = True
)

In [21]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_set,
    eval_dataset = val_set,
    dataset_text_field = 'messages',
    max_seq_length = 2048, 
    peft_config = peft_config, 
    packing=False,
    args = training_arguments,
    callbacks=[early_stopping_callback]
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/928 [00:00<?, ? examples/s]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [ ]:
%%time

eot = "<|eot_id|>"
eot_id = tokenizer.convert_tokens_to_ids(eot)
tokenizer.pad_token = eot
tokenizer.pad_token_id = eot_id

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss


In [ ]:
adapter_model = new_model+'_adapter'
adapter_model

In [ ]:
trainer.model.save_pretrained(adapter_model)
trainer.model.push_to_hub(adapter_model, use_temp_dir=False)

In [ ]:
adapter_model = 'huyhoangt2201/' + adapter_model
base_model = 'phamhai/Llama-3.2-1B-Instruct-Frog'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

# base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)

# Merge adapter with base model
merge_model = PeftModel.from_pretrained(base_model_reload, adapter_model)

merge_model = merge_model.merge_and_unload()

In [ ]:
merged_model = new_model + '_merged'
merge_model.save_pretrained(merged_model)
tokenizer.save_pretrained(merged_model)

In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HF_TOKEN")
login(token = hf_token)

In [ ]:
merge_model.push_to_hub(merged_model, use_temp_dir=False)
tokenizer.push_to_hub(merged_model, use_temp_dir=False)